In [1]:
import findspark
findspark.init('/home/osboxes/spark-2.4.3-bin-hadoop2.7')
import pyspark
import os

In [3]:
myPath = os.path.join('/home', 'osboxes', 'CourseMaterial', 'Spark_for_Machine_Learning', 'Recommender_Systems')

print(myPath)

os.chdir(myPath)

/home/osboxes/CourseMaterial/Spark_for_Machine_Learning/Recommender_Systems


In [4]:
os.listdir()

['movielens_ratings.csv',
 'Consulting Project - Recommender Systems.ipynb',
 '.ipynb_checkpoints',
 'Recommender_Code_Along.ipynb',
 'Meal_Info.csv']

In [5]:
# set up the session info (Alternating Least Squares)

from pyspark.sql import SparkSession

name = 'collabFilter'

spark = SparkSession.builder.appName(name).getOrCreate()

from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [6]:
# use the movie lens data

data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [9]:
# split the data

training, test = data.randomSplit([0.8, 0.2])

In [12]:
als = ALS(maxIter=5, regParam=0.01,userCol='userId', itemCol='movieId', ratingCol='rating')

In [13]:
model = als.fit(training)

In [14]:
# negative is a real dislike for the movie

predictions = model.transform(test)
predictions.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   4.0|    12|0.31941757|
|     31|   3.0|     8| 1.5068874|
|     85|   3.0|     1| -1.317255|
|     85|   1.0|    13|   1.92203|
|     65|   1.0|    22| 2.0380826|
|     65|   2.0|     5|  0.613713|
|     65|   1.0|     4| 0.9369513|
|     65|   5.0|    23| 0.7929954|
|     53|   1.0|    12|  3.596769|
|     53|   2.0|    19| 1.5840188|
|     81|   1.0|    16|0.14966965|
|     81|   1.0|     7| 2.3556554|
|     28|   3.0|     1| 1.9695637|
|     76|   1.0|    26| 2.0672503|
|     76|   1.0|    11|0.83807003|
|     76|   1.0|     2| 2.8955843|
|     26|   1.0|    22| 2.8439445|
|     26|   1.0|    20| 0.2556646|
|     26|   1.0|    18| 0.7264143|
|     27|   1.0|    28|0.77665544|
+-------+------+------+----------+
only showing top 20 rows



In [15]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating',
                               predictionCol='prediction')

In [16]:
# check the scale of the error relative to 5

rmse = evaluator.evaluate(predictions)
print('RMSE')
print(rmse)

RMSE
2.050136334282802


In [17]:
# get a single user

single_user = test.filter(test['userId']==11).select(['movieId', 'userId'])
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     40|    11|
|     61|    11|
|     66|    11|
|     67|    11|
|     69|    11|
|     76|    11|
|     94|    11|
+-------+------+



In [18]:
recommendations = model.transform(single_user)

In [19]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     66|    11| 2.7259479|
|     94|    11| 1.8423632|
|     40|    11| 1.8077023|
|     67|    11| 1.3993847|
|     61|    11|0.92279124|
|     76|    11|0.83807003|
|     69|    11|0.17981708|
|      9|    11|-0.7677968|
+-------+------+----------+

